# HW3.  Database Design

### Library Relations

- Member = {<span style="text-decoration:underline">memberID</span>, name, birthday, status}
- Borrow = {<span style="text-decoration:underline">borrowID</span><sup>FK-BorrowTransactions</sup>, <span style="text-decoration:underline">memberID</span><sup>FK-Member</sup>,<span style="text-decoration:underline">itemID</span><sup>FK-Item</sup>}
- BorrowTransactions = {<span style="text-decoration:underline">borrowID</span>, borrowDate, returnDate}
- Owes = {<span style="text-decoration:underline">fineID</span><sup>FK-Fines</sup>,<span style="text-decoration:underline">memberID</span><sup>FK-Member</sup>,<span style="text-decoration:underline">itemID</span><sup>FK-Item</sup>}
- Fines = {<span style="text-decoration:underline">fineID</span>, status, amount}
- Item = {<span style="text-decoration:underline">itemID</span>, name, author, category, genre, status}
- Records = {<span style="text-decoration:underline">itemID</span><sup>FK-Item</sup>, status, lastUpdated}
- Volunteers = {<span style="text-decoration:underline">memberID</span><sup>FK-Member</sup>,<span style="text-decoration:underline">staffID</span>}
- Staff = {<span style="text-decoration:underline">memberID</span><sup>FK-Member</sup>, <span style="text-decoration:underline">staffID</span>, position, wage, employmentDate}
- Volunteer = {<span style="text-decoration:underline">memberID</span><sup>FK-Member</sup>, <span style="text-decoration:underline">volunteerID</span>, employmentDate}
- Hold = {{<span style="text-decoration:underline">staffID</span><sup>FK-Staff</sup>>,<span style="text-decoration:underline">eventID</span>}<sup>FK-Events</sup>}
- Events = {<span style="text-decoration:underline">eventID</span>, name, scheduledTime, scheduledDate, targetAudience}
- Located = {<span style="text-decoration:underline">eventID</span><sup>FK-Events</sup>, <span style="text-decoration:underline">roomNum</span><sup>FK-Room</sup>}
- Room = {<span style="text-decoration:underline">roomNum</span>, maxCap}

#### Creating Database

In [184]:
%load_ext sql
%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

In [185]:
%sql sqlite:///library.db

In [186]:
%%sql

CREATE TABLE Member (
    memberID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    birthday DATE,
    status VARCHAR(8) CHECK (status IN ('Active', 'Inactive'))
);

CREATE TABLE Borrow (
    borrowID INTEGER PRIMARY KEY,
    memberID INTEGER,
    itemID INTEGER,
    FOREIGN KEY (borrowID) REFERENCES BorrowTransactions(borrowID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);


CREATE TABLE BorrowTransactions (
    borrowID INTEGER PRIMARY KEY,
    borrowDate DATE,
    returnDate DATE
);

CREATE TABLE Owes (
    fineID INTEGER PRIMARY KEY,
    memberID INTEGER,
    itemID INTEGER,
    FOREIGN KEY (fineID) REFERENCES Fines(fineID),
    FOREIGN KEY (memberID) REFERENCES Member(memberID),
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);

CREATE TABLE Fines (
    fineID INTEGER PRIMARY KEY,
    status VARCHAR(6) CHECK (status IN ('Paid', 'Unpaid')),
    amount DECIMAL(10,2) CHECK (amount >= 0)
);

CREATE TABLE Item (
    itemID INTEGER PRIMARY KEY,
    name VARCHAR(255),
    author VARCHAR(100),
    category VARCHAR(50),
    genre VARCHAR(50),
    status VARCHAR(11) CHECK (status IN ('Available', 'Unavailable'))
);

CREATE TABLE Records (
    itemID INTEGER,
    status VARCHAR(20) CHECK (status IN ('In System', 'Awaiting Approval', 'Archived')),
    lastUpdated DATE,
    FOREIGN KEY (itemID) REFERENCES Item(itemID)
);

CREATE TABLE Staff (
    memberID INTEGER,
    staffID INTEGER PRIMARY KEY,
    position VARCHAR(50),
    wage DECIMAL(10,2),
    employmentDate DATE,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

CREATE TABLE Volunteer (
    memberID INTEGER,
    volunteerID INTEGER PRIMARY KEY,
    employmentDate DATE,
    FOREIGN KEY (memberID) REFERENCES Member(memberID)
);

CREATE TABLE Hold (
    staffID INTEGER,
    eventID INTEGER,
    PRIMARY KEY (staffID, eventID),
    FOREIGN KEY (staffID) REFERENCES Staff(staffID),
    FOREIGN KEY (eventID) REFERENCES Events(eventID)
);

CREATE TABLE Events (
    eventID INTEGER PRIMARY KEY,
    name VARCHAR(100),
    scheduledTime TIME,
    scheduledDate DATE,
    targetAudience VARCHAR(100)
);

CREATE TABLE Located (
    eventID INTEGER,
    roomNum INTEGER,
    PRIMARY KEY (eventID, roomNum),
    FOREIGN KEY (eventID) REFERENCES Events(eventID),
    FOREIGN KEY (roomNum) REFERENCES Room(roomNum)
);

CREATE TABLE Room (
    roomNum INTEGER PRIMARY KEY,
    maxCap INTEGER
);

Running query in 'sqlite:///library.db'

++
||
++
++

#### Triggers

1) Aborts process if user attempts to borrow item while they have unpaid fines

In [187]:
%%sql
CREATE TRIGGER prevent_borrow_if_fine_unpaid
BEFORE INSERT ON Borrow
FOR EACH ROW
BEGIN
    -- Check if the member has an unpaid fine
    SELECT RAISE(ABORT, 'Cannot borrow item. Member has unpaid fines.')
    FROM Fines
    JOIN Owes ON Fines.fineID = Owes.fineID
    WHERE Owes.memberID = NEW.memberID AND Fines.status = 'Unpaid'
    LIMIT 1;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

2) Prevent item deletion if it is still borrowed

In [188]:
%%sql
CREATE TRIGGER prevent_item_deletion_if_borrowed
BEFORE DELETE ON Item
FOR EACH ROW
BEGIN
    -- Prevent item deletion if it is still borrowed
    SELECT RAISE(ABORT, 'Item cannot be deleted as it is currently borrowed.')
    WHERE EXISTS (SELECT 1 FROM Borrow WHERE itemID = OLD.itemID);
END;


Running query in 'sqlite:///library.db'

++
||
++
++

3. When add item, also adds record

In [189]:
%%sql

CREATE TRIGGER add_record_on_item_insert
AFTER INSERT ON Item
FOR EACH ROW
BEGIN
    INSERT INTO Records (itemID, status, lastUpdated)
    VALUES (NEW.itemID, 'Awaiting Approval', DATE('now'));
END;

Running query in 'sqlite:///library.db'

++
||
++
++

# also need: 
- if item not in system cant borrow

Sets ids so they increase

In [190]:
%%sql
CREATE TRIGGER set_member_id
BEFORE INSERT ON Member
FOR EACH ROW
WHEN NEW.memberID IS NULL
BEGIN
    UPDATE Member
    SET memberID = (
        CASE 
            WHEN (SELECT MAX(memberID) FROM Member) IS NULL THEN 1
            ELSE (SELECT MAX(memberID) FROM Member) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

CREATE TRIGGER set_borrow_transactions
BEFORE INSERT ON BorrowTransactions
FOR EACH ROW
WHEN NEW.borrowID IS NULL
BEGIN
    UPDATE BorrowTransactions
    SET borrowID = (
        CASE 
            WHEN (SELECT MAX(borrowID) FROM BorrowTransactions) IS NULL THEN 1
            ELSE (SELECT MAX(borrowID) FROM BorrowTransactions) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

CREATE TRIGGER set_fines
BEFORE INSERT ON Fines
FOR EACH ROW
WHEN NEW.fineID IS NULL
BEGIN
    UPDATE Fines
    SET fineID = (
        CASE 
            WHEN (SELECT MAX(fineID) FROM Fines) IS NULL THEN 1
            ELSE (SELECT MAX(fineID) FROM Fines) + 1
        END
    )
    WHERE rowid = NEW.rowid;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [191]:
%%sql
PRAGMA table_info(Fines)

Running query in 'sqlite:///library.db'

cid,name,type,notnull,dflt_value,pk
0,fineID,INTEGER,0,None,1
1,status,VARCHAR(6),0,None,0
2,amount,"DECIMAL(10,2)",0,None,0


#### Inserts

In [192]:
%%sql
-- Insert data into Member
INSERT INTO Member (memberID, name, birthday, status) VALUES
(1110, 'Elena Nguyen', '2005-01-15', 'Active'),
(NULL, 'Nathan Thamtoro', '2004-01-22', 'Active'),
(NULL, 'Charlie Brown', '2000-07-09', 'Inactive'),
(NULL, 'Bob Ross', '1995-12-30', 'Active'),
(NULL, 'Ethan Hunt', '1992-08-18', 'Inactive'),
(NULL, 'Fiona Apple', '1998-06-25', 'Active'),
(NULL, 'George Lucas', '1980-11-05', 'Active'),
(NULL, 'Helen Keller', '1975-05-14', 'Inactive'),
(NULL, 'Isaac Newton', '1963-04-01', 'Active'),
(NULL, 'Jack Sparrow', '1989-09-19', 'Active');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [193]:
%%sql
-- BorrowTransactions
INSERT INTO BorrowTransactions (borrowID, borrowDate, returnDate) VALUES
(2220, '2024-01-01', '2024-01-15'),
(NULl, '2024-01-05', '2024-01-20'),
(NULl, '2024-01-10', '2024-01-25'),
(NULl, '2024-01-12', '2024-01-27'),
(NULl, '2024-01-15', '2024-01-30'),
(NULl, '2024-01-20', '2024-02-05'),
(NULl, '2024-01-25', '2024-02-10'),
(NULl, '2024-02-01', '2024-02-15'),
(NULl, '2024-02-05', '2024-02-20'),
(NULl, '2024-02-10', '2024-02-25');


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [194]:
%%sql
--Fines
INSERT INTO Fines (fineID, status, amount) VALUES
(3330, 'Unpaid', 5.00),
(NULl, 'Paid', 5.00),
(NULl, 'Unpaid', 10.00),
(NULl, 'Paid', 7.50),
(NULl, 'Unpaid', 15.00),
(NULl, 'Paid', 12.00),
(NULl, 'Unpaid', 20.00),
(NULl, 'Paid', 8.00),
(NULl, 'Unpaid', 25.00),
(NULl, 'Paid', 15.00);


Running query in 'sqlite:///library.db'

10 rows affected.

++
||
++
++

In [195]:
%%sql
--Items
INSERT INTO Item (itemID, name, author, category, genre, status) VALUES
(4440, 'Harry Potter and the Sorcerer''s Stone', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'Harry Potter and the Chamber of Secrets', 'J.K. Rowling', 'Book', 'Fantasy', 'Unavailable'),
(NULL, 'Harry Potter and the Prisoner of Azkaban', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'The Hobbit', 'J.R.R. Tolkien', 'Book', 'Fantasy', 'Available'),
(NULL, '1984', 'George Orwell', 'Book', 'Dystopian', 'Unavailable'),
(NULL, 'To Kill a Mockingbird', 'Harper Lee', 'Book', 'Classic', 'Available'),
(NULL, 'The Catcher in the Rye', 'J.D. Salinger', 'Book', 'Classic', 'Unavailable'),
(NULL, 'Pride and Prejudice', 'Jane Austen', 'Book', 'Romance', 'Available'),
(NULL, 'Rock Legends Collection', 'Various Artists', 'CD', 'Rock', 'Available'),
(NULL, 'Jazz Classics', 'Various Artists', 'CD', 'Jazz', 'Unavailable');

Running query in 'sqlite:///library.db'

RuntimeError: (sqlite3.OperationalError) foreign key mismatch - "History" referencing "Records"
[SQL: INSERT INTO Item (itemID, name, author, category, genre, status) VALUES
(4440, 'Harry Potter and the Sorcerer''s Stone', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'Harry Potter and the Chamber of Secrets', 'J.K. Rowling', 'Book', 'Fantasy', 'Unavailable'),
(NULL, 'Harry Potter and the Prisoner of Azkaban', 'J.K. Rowling', 'Book', 'Fantasy', 'Available'),
(NULL, 'The Hobbit', 'J.R.R. Tolkien', 'Book', 'Fantasy', 'Available'),
(NULL, '1984', 'George Orwell', 'Book', 'Dystopian', 'Unavailable'),
(NULL, 'To Kill a Mockingbird', 'Harper Lee', 'Book', 'Classic', 'Available'),
(NULL, 'The Catcher in the Rye', 'J.D. Salinger', 'Book', 'Classic', 'Unavailable'),
(NULL, 'Pride and Prejudice', 'Jane Austen', 'Book', 'Romance', 'Available'),
(NULL, 'Rock Legends Collection', 'Various Artists', 'CD', 'Rock', 'Available'),
(NULL, 'Jazz Classics', 'Various Artists', 'CD', 'Jazz', 'Unava

In [ ]:
%%sql
SELECT * FROM Item;

#### Indexes

In [ ]:
%%sql

CREATE INDEX ItemIndex ON Item(name, author);

#### FOR TESTING PURPOSES
Delete tables if you need to

In [183]:
%%sql 
-- Disable foreign key checks temporarily
PRAGMA foreign_keys = OFF;

-- Drop all tables (starting with those with foreign keys)
DROP TABLE IF EXISTS Hold;
DROP TABLE IF EXISTS Located;
DROP TABLE IF EXISTS Events;
DROP TABLE IF EXISTS Staff;
DROP TABLE IF EXISTS Volunteer;
DROP TABLE IF EXISTS Borrow;
DROP TABLE IF EXISTS BorrowTransactions;
DROP TABLE IF EXISTS Records;
DROP TABLE IF EXISTS Fines;
DROP TABLE IF EXISTS Owes;
DROP TABLE IF EXISTS Item;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Room;

-- Re-enable foreign key checks
PRAGMA foreign_keys = ON;


Running query in 'sqlite:///library.db'

++
||
++
++